In [1]:
import numpy as np
import pandas as pd

In [49]:
import geopandas as gpd

In [2]:
import os

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import sys
sys.path.insert(1, '../tools')
from data_tools import *

In [50]:
import re

# Business Info DataFrame

In [7]:
business_info = pd.read_csv('../data/all_businesses_info.csv')

In [59]:
in_str = ' SAN FRANCISCO  '

In [75]:
in_str.strip()

'SAN FRANCISCO'

In [74]:
re.sub(r'^\s+|\s+$', '', in_str)

'SAN FRANCISCO'

In [8]:
business_info['city'] = business_info

,id,alias,name,image_url,is_closed,url,review_count,rating,price,phone,...,transactions2,transactions3,address1,address2,address3,city,zip_code,country,state,display_address
0,ucXuxH-JOwyL_hi0uzXjVw,lovina-calistoga,Lovina,https://s3-media1.fl.yelpcdn.com/bphoto/fiMg4W...,False,https://www.yelp.com/biz/lovina-calistoga?adju...,327,4.0,$$,1.707943e+10,...,NaN,NaN,1107 Cedar St,NaN,NaN,Calistoga,94515.0,US,CA,"1107 Cedar St, Calistoga, CA 94515"
1,U69SHL0P_DGbjaB8TNIwLg,sams-social-club-calistoga-2,Sam's Social Club,https://s3-media1.fl.yelpcdn.com/bphoto/uzjx9K...,False,https://www.yelp.com/biz/sams-social-club-cali...,640,4.0,$$,1.707942e+10,...,NaN,NaN,1712 Lincoln Ave,NaN,NaN,Calistoga,94515.0,US,CA,"1712 Lincoln Ave, Calistoga, CA 94515"
2,XITkuNFTKXeDKotFD9dPxA,solbar-calistoga,SolBar,https://s3-media1.fl.yelpcdn.com/bphoto/_5YbbL...,False,https://www.yelp.com/biz/solbar-calistoga?adju...,1047,4.0,$$$,1.707226e+10,...,NaN,NaN,755 Silverado Trl,NaN,Solage Hotel,Calistoga,94515.0,US,CA,"755 Silverado Trl, Solage Hotel, Calistoga, CA..."
3,41jZtOex0ITMwM353GYUFw,evangeline-calistoga,Evangeline,https://s3-media1.fl.yelpcdn.com/bphoto/fkquwF...,False,https://www.yelp.com/biz/evangeline-calistoga?...,375,4.0,$$,1.707341e+10,...,NaN,NaN,1226 Washington St,NaN,NaN,Calistoga,94515.0,US,CA,"1226 Washington St, Calistoga, CA 94515"
4,yPLq0SrnSnkTH1jQfgVMTA,calistoga-inn-calistoga-7,Calistoga Inn,https://s3-media1.fl.yelpcdn.com/bphoto/BRpXx4...,False,https://www.yelp.com/biz/calistoga-inn-calisto...,611,4.0,$$,1.707942e+10,...,NaN,NaN,1250 Lincoln Ave,NaN,NaN,Calistoga,94515.0,US,CA,"1250 Lincoln Ave, Calistoga, CA 94515"


In [54]:
in_str = 'SAN FRANCISCO'

In [55]:
in_str.capitalize()

'San francisco'

In [9]:
business_info[['latitude', 'longitude']].head()

,latitude,longitude
0,38.576590,-122.579640
1,38.582206,-122.575248
2,38.585059,-122.570257
3,38.579050,-122.579480
4,38.576928,-122.579623


In [23]:
geo_xy = gpd.points_from_xy(business_info.longitude, business_info.longitude)
geo_business_info = gpd.GeoDataFrame(business_info, geometry=geo_xy)

In [44]:
val, count = np.unique(geo_business_info.city.values, return_counts=True)

In [46]:
val

array(['ALAMEDA', 'Alameda', 'Albany', 'Alum Rock', 'American Canyon',
       'Antioch', 'Ashland', 'Bay Area', 'Belmont', 'Belvedere Tiburon',
       'Benicia', 'Berkeley', 'Brentwood', 'Brentwood Ca', 'Brisbane',
       'Burlingame', 'CA', 'Calistoga', 'Campbell', 'Castro Valley',
       'Clayton', 'Cloverdale', 'Colma', 'Concord', 'Corte Madera',
       'Cotati', 'Cupertino', 'Daly City', 'Danville', 'Dixon', 'Dublin',
       'East Palo Alto', 'El Cerrito', 'El Sobrante', 'Elk Grove',
       'Emeryville', 'Fairfax', 'Fairfield', 'Fairfield Ca',
       'Foster City', 'Fremont', 'Gilroy', 'Gilroy Ca', 'Greenbrae',
       'Hacienda Heights', 'Half Moon Bay', 'Hayward', 'Healdsburg',
       'Hercules', 'Hillsborough', 'Ignacio', 'Kensington', 'Kentfield',
       'Knob Noster MO', 'Lafayette', 'Larkspur', 'Las Vegas',
       'Livermore', 'Los Altos', 'Los Atlos', 'Los Gatos', 'Martinez',
       'Menlo Park', 'Mill Valley', 'Millbrae', 'Milpitas',
       'Moffett Field', 'Moraga', 'Morgan

In [45]:
count

array([   1,  364,  131,   23,   27,  164,    2,    2,  137,    4,   46,
        619,  131,    2,   16,  251,    2,   24,  141,   78,    8,   18,
         32,  423,   37,   61,  155,  228,   66,   20,  248,   46,  143,
         31,    2,  113,   26,  183,    2,   43,  736,  131,    1,    9,
          2,   56,  459,   52,   27,    2,    1,    3,    1,   19,   50,
         14,   11,  186,  109,    2,  159,   86,  174,   35,   90,  283,
          1,   26,  107,  330,  275,  383,    1,  111,   19, 1241,   29,
         27,    1,    7,   58,  401,  150,    5,   74,  124,  172,  290,
          2,    1,    1,    9,    1,  350,    3,  203,   17,    3,  110,
          6,    1,    1,  115,   23,    1,   26,  202,  205, 1448, 1541,
         33,  307,   43,  403,  143,  205,  164,    3,  537,   43,  376,
          9,   37,   30,   47,   58,   79,    1,   17,   45,  131,   78,
        517,    2,   19,   35,    5,  205,  193,    1,  249,  275,    2,
         52,    7,   22,    1])